In [51]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from matplotlib import cm 
from mpl_toolkits.mplot3d import axes3d, Axes3D 
from IPython.display import display
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper, Range1d
from bokeh.palettes import Category10_5, Category20_16
from bokeh.io import export_svgs
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit
import scipy    
from scipy.interpolate import interp1d
from scipy import interpolate
from scipy.ndimage import gaussian_filter
from pylatexenc.latex2text import LatexNodes2Text
import svglib.svglib as svglib
from reportlab.graphics import renderPDF
import csv
%matplotlib qt5

# Variables

## Parameters

In [2]:
photon_energy = [1.550, 1.569, 1.600, 1.610, 1.631, 1.653, 1.675, 1.698, 1.722, 1.746, 1.771, 1.797, 1.823, 1.851, 1.879, 1.907, 1.937, 1.968, 2.000, 2.033, 2.066, 2.101, 2.138, 2.175, 2.214, 2.254, 2.296, 2.339, 2.384, 2.431, 2.480, 2.530, 2.583, 2.638, 2.695, 2.755, 2.818, 2.883, 2.952, 3.024, 3.100, 3.179, 3.263, 3.351, 3.444, 3.542, 3.647, 3.757, 3.875, 3.999, 4.133, 4.275, 4.428, 4.592, 4.769, 4.959, 5.166, 5.391, 5.636, 5.904, 6.199]
energy_array = np.array(photon_energy, dtype=float)

absorption = [0.483, 0.463, 0.424, 0.398, 0.392, 0.405, 0.420, 0.556, 0.855, 1.192, 1.538, 2.000, 2.222, 2.326, 2.500, 2.865, 3.040, 3.135, 3.236, 3.460, 4.098, 6.369, 9.259, 12.516, 14.124, 15.674, 17.921, 19.724, 20.964, 28.011, 38.911, 51.020, 56.818, 64.103, 64.103, 68.966, 68.966, 69.444, 65.359, 61.728, 58.480, 52.356, 45.455, 33.333, 26.385, 21.598, 17.825, 14.749, 11.848, 9.524, 7.092, 4.651, 3.472, 2.681, 2.188, 1.789, 1.389, 1.079, 0.763, 0.503, 0.326]
absorption_array = np.array(absorption)

mie_scattering = [2500.000, 2500.000, 2500.000, 2000.000, 2000.000, 2000.000, 1666.667, 1666.667, 1666.667, 1428.571, 1428.571, 1428.571, 1428.571, 1250.000, 1250.000, 1000.000, 1000.000, 909.091, 833.333, 769.231, 714.286, 666.667, 625.000, 588.235, 555.556, 526.316, 476.190, 454.545, 416.667, 384.616, 344.828, 322.581, 294.118, 270.270, 243.902, 222.222, 204.082, 181.818, 163.934, 147.059, 131.579, 119.048, 106.383, 94.340, 83.333, 74.627, 65.359, 57.143, 50.000, 43.668, 38.168, 32.787, 28.329, 24.096, 20.619, 17.391, 14.599, 12.195, 10.050, 8.403, 6.623]
mie_scat_array = np.array(mie_scattering)

glass_index = [1.5074, 1.5075, 1.5075, 1.5076, 1.5083, 1.5091, 1.5097, 1.5099, 1.5099, 1.5105, 1.5101, 1.5105, 1.5104, 1.5108, 1.5110, 1.5112, 1.5116, 1.5116, 1.5114, 1.5118, 1.5135, 1.5140, 1.5141, 1.5145, 1.5152, 1.5153, 1.5163, 1.5174, 1.5176, 1.5182, 1.5187, 1.5193, 1.5201, 1.5206, 1.5217, 1.5222, 1.5231, 1.5246, 1.5257, 1.5261, 1.5278, 1.5292, 1.5314, 1.5336, 1.5365, 1.5394, 1.5417, 1.5442, 1.5468, 1.5491, 1.5532, 1.5566, 1.5626, 1.5670, 1.5718, 1.5790, 1.5870, 1.5957, 1.6043, 1.6232, 1.6371]
glass_index_array = np.array(glass_index)

glass_abs = [316.9572108, 316.9572108, 316.9572108, 316.9572108, 316.9572108, 316.9572108, 316.9572108, 316.9572108, 304.5066991, 274.2731761, 240.0384061, 206.6542674, 181.8181818, 156.3774502, 139.3444862, 129.1035274, 121.9334902, 119.7135326, 122.1887636, 127.2367858, 133.4467992, 141.0302622, 150.3753298, 161.8315483, 177.9342304, 161.8315483, 151.6952017, 143.7850612, 135.0713139, 125.22856, 113.0143301, 104.9198502, 95.44620406, 82.7502968, 67.23444119, 58.77205598, 51.38245418, 41.81366949, 29.22220534, 21.66861048, 12.8737616, 7.079034837, 3.023689376, 1.413360718, 0.5345729838, 0.1552022043, 0.04916730433, 0.0199758852, 0.01166419596, 0.01034027868, 0.009265061783, 0.008861001454, 0.007978840712, 0.007699001607, 0.007434528078, 0.007098095188, 0.007204924393, 0.008051878933, 0.009117465747, 0.01042013539]
glass_abs_array = np.array(glass_abs)

In [3]:
index_energy = [1.926, 2.104, 2.141, 2.149, 2.270, 2.438, 2.583, 2.650, 2.845, 3.064]
index_energy_array = np.array(index_energy)

index = [1.35072, 1.34811, 1.34596, 1.34525, 1.34378, 1.34215, 1.34104, 1.34097, 1.34063, 1.33905]
index_array = np.array(index)

In [4]:
ray_energy = [2.066, 2.156, 2.254, 2.362, 2.480, 2.610, 2.755, 2.917, 3.100, 3.306, 3.542]
ray_array = np.array(ray_energy, dtype=float)

ray_scattering = [(1/(14.1*10**(-4))), (1/(16.2*10**(-4))), (1/(19.3*10**(-4))), (1/(23.3*10**(-4))), (1/(28.8*10**(-4))), (1/(35.9*10**(-4))), (1/(45.4*10**(-4))), (1/(58.1*10**(-4))), (1/(75.5*10**(-4))), (1/(99.8*10**(-4))), (1/(134.5*10**(-4)))]
ray_scat_array = np.array(ray_scattering, dtype=float)
print(ray_scat_array)

print(type(ray_array), type(ray_array[0]))
print(type(ray_scat_array), type(ray_scat_array[0]))
print(type(photon_energy), type(photon_energy[0]))

[709.21985816 617.28395062 518.13471503 429.18454936 347.22222222
 278.55153203 220.26431718 172.11703959 132.45033113 100.2004008
  74.34944238]
<class 'numpy.ndarray'> <class 'numpy.float64'>
<class 'numpy.ndarray'> <class 'numpy.float64'>
<class 'list'> <class 'float'>


In [5]:
interp_func_1 = interp1d(index_energy, index, kind='cubic', fill_value="extrapolate")
index_interp = interp_func_1(photon_energy)
print(index_interp)

[1.64258552 1.6088308  1.55944857 1.54495862 1.51670815 1.49014426
 1.46651484 1.44476393 1.42507754 1.40824721 1.39351408 1.38095547
 1.37093717 1.36266646 1.3566642  1.352579   1.3499301  1.34862199
 1.348295   1.34849721 1.3486777  1.34820405 1.34621263 1.34365331
 1.34305765 1.34355651 1.3439364  1.34370239 1.34305238 1.34225758
 1.34162605 1.34123825 1.34104    1.34097687 1.34093704 1.34086058
 1.34071744 1.34047485 1.3400799  1.33948018 1.33859649 1.33735626
 1.33562047 1.33327433 1.33012976 1.3259849  1.32048673 1.31342383
 1.30420789 1.29250953 1.27731959 1.25805908 1.23332341 1.20180281
 1.16144802 1.11016025 1.04406424 0.95907282 0.84958298 0.70780452
 0.52279934]


In [6]:
# Define exponential model
def exp_func(x, a, b):
    return a * np.exp(b * x)

# Fit exponential model to your data
popt, _ = curve_fit(exp_func, ray_array, ray_scat_array)

# Apply the fitted model to photon_energy
ray_interp = exp_func(energy_array, *popt)
print(ray_interp)

[1.67267108e+03 1.62058204e+03 1.53905415e+03 1.51363991e+03
 1.46162774e+03 1.40905470e+03 1.35837265e+03 1.30733494e+03
 1.25612158e+03 1.20691445e+03 1.15770567e+03 1.10865573e+03
 1.06168394e+03 1.01332212e+03 9.67163284e+02 9.23107072e+02
 8.78128533e+02 8.33951831e+02 7.90679923e+02 7.48406108e+02
 7.08392469e+02 6.68288952e+02 6.28359755e+02 5.90816263e+02
 5.53669061e+02 5.17994247e+02 4.83006919e+02 4.49633472e+02
 4.17174400e+02 3.85771732e+02 3.55546891e+02 3.27144962e+02
 2.99511974e+02 2.73301412e+02 2.48555453e+02 2.24923751e+02
 2.02524673e+02 1.81749949e+02 1.62023542e+02 1.43718458e+02
 1.26635201e+02 1.11026574e+02 9.65347704e+01 8.33773459e+01
 7.14161938e+01 6.06638080e+01 5.09331700e+01 4.24088094e+01
 3.48438703e+01 2.83437886e+01 2.26755632e+01 1.79008313e+01
 1.38750276e+01 1.05594196e+01 7.86403635e+00 5.73126180e+00
 4.06033365e+00 2.79162323e+00 1.85647551e+00 1.18820011e+00
 7.27051814e-01]


In [7]:
phi = [0.1, 0.126, 0.158, 0.2, 0.251, 0.316, 0.398, 0.501, 0.631, 0.794, 1, 1.259, 1.585, 1.995, 2.512, 3.162, 3.981, 5.012, 6.310, 7.943, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180]
vol_scat = [2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.936 * 10**(-4), 2.935 * 10**(-4), 2.935 * 10**(-4), 2.934 * 10**(-4), 2.933 * 10**(-4), 2.932 * 10**(-4), 2.930 * 10**(-4), 2.926 * 10**(-4), 2.920 * 10**(-4), 2.911 * 10**(-4), 2.896 * 10**(-4), 2.847 * 10**(-4), 2.780 * 10**(-4), 2.697 * 10**(-4), 2.602 * 10**(-4), 2.497 * 10**(-4), 2.384 * 10**(-4), 2.268 * 10**(-4), 2.152 * 10**(-4), 2.040 * 10**(-4), 1.934 * 10**(-4), 1.839 * 10**(-4), 1.756 * 10**(-4), 1.690 * 10**(-4), 1.640 * 10**(-4), 1.610 * 10**(-4), 1.600 * 10**(-4), 1.610 * 10**(-4), 1.640 * 10**(-4), 1.690 * 10**(-4), 1.756 * 10**(-4), 1.839 * 10**(-4), 1.934 * 10**(-4), 2.040 * 10**(-4), 2.152 * 10**(-4), 2.268 * 10**(-4), 2.384 * 10**(-4), 2.497 * 10**(-4), 2.602 * 10**(-4), 2.697 * 10**(-4), 2.780 * 10**(-4), 2.847 * 10**(-4), 2.896 * 10**(-4), 2.929 * 10**(-4), 2.936 * 10**(-4)]

## Oscilloscope

In [49]:
time = []
voltage = []

with open('/Users/lgm/Desktop/Geant4/Oscilloscope_Data.csv') as oscilloscope_data:
    plots = csv.reader(oscilloscope_data, delimiter = ',')

    for row in plots:
        time.append(float(row[0]))
        voltage.append(float(row[1]))

In [63]:
k = 0.65

smooth_voltage = gaussian_filter(voltage,k)

## ROOT

### Accessing Trees and Leaves

In [8]:
def readinG4rootfile(filename):
    try:
        # Open the ROOT datafile
        datafile = uproot.open(filename)
 
        # Access trees
        primarytree = datafile["primarytree"]
        steptree = datafile["steptree"]
 
        # Read specific branches from all_events
        primary_df = primarytree.arrays([
            "PrimaryX0_mm",
            "PrimaryY0_mm",
            "PrimaryZ0_mm",
            "primaryenergy_eV",
            "primaryinfo_PDGID",
            "primarypolarization_X",
            "primarypolarization_Y",
            "primarypolarization_Z"],
            library="pd")
   
        # Read argon_steps branches
        step_df = steptree.arrays([
            "Stepenergy_eV",
            "Stepinfo_PDGID",
            "Stepinfo_ParentID",
            "Stepinfo_eventnumber",
            "Stepinfo_endoftrackflag",
            "Stepposition_X_mm",
            "Stepposition_Y_mm",
            "Stepposition_Z_mm",
            "Stepinfo_polarization_X",
            "Stepinfo_polarization_Y",
            "Stepinfo_polarization_Z"],
            library="pd")
 
        return primary_df,step_df
 
    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
        return None

In [9]:
pony_file = '/Users/lgm/g4test/Outputs/PONY_Output/PONYsimdebugging.root'
pony_file_content = readinG4rootfile(pony_file)
pony_data = pony_file_content[1]
print(pony_data)
event = pony_data["Stepinfo_endoftrackflag"]

         Stepenergy_eV  Stepinfo_PDGID  Stepinfo_ParentID  \
0                  0.0           -22.0                  0   
1                  0.0           -22.0                  0   
2                  4.0           -22.0                  0   
3                  4.0           -22.0                  0   
4                  4.0           -22.0                  0   
...                ...             ...                ...   
2806516            4.0           -22.0                  0   
2806517            0.0           -22.0                  0   
2806518            0.0           -22.0                  0   
2806519            4.0           -22.0                  0   
2806520            4.0           -22.0                  0   

         Stepinfo_eventnumber  Stepinfo_endoftrackflag  Stepposition_X_mm  \
0                           0                    False       -2667.963894   
1                           0                    False       -2667.963894   
2                           0       

### Position

In [10]:
pony_data["distances"] = np.sqrt(pony_data["Stepposition_X_mm"]**2 + pony_data["Stepposition_Y_mm"]**2 + pony_data["Stepposition_Z_mm"]**2)
pony_data = pony_data[pony_data["distances"] > 0]

positions_x_mm = pony_data["Stepposition_X_mm"]
positions_y_mm = pony_data["Stepposition_Y_mm"]
positions_z_mm = pony_data["Stepposition_Z_mm"]

positions_x_min = positions_x_mm.min()
positions_x_max = positions_x_mm.max()
positions_y_min = positions_y_mm.min()
positions_y_max = positions_y_mm.max()
positions_z_min = positions_z_mm.min()
positions_z_max = positions_z_mm.max()

print(positions_x_min)
print(positions_x_max)
print(positions_y_min)
print(positions_y_max)
print(positions_z_min)
print(positions_z_max)

multi_positions_bins = (positions_y_max - positions_z_min)/100
print(multi_positions_bins)

-48000.0
48000.0
-48000.0
48000.0
-48000.0
48000.0
960.0


In [11]:
mask = (event >= 1)
x_pos_filtered = positions_x_mm[mask]
y_pos_filtered = positions_y_mm[mask]
z_pos_filtered = positions_z_mm[mask]

print(len(positions_x_mm))
print(len(x_pos_filtered))
print(mask)
print(event)

2806521
1000000
0          False
1          False
2          False
3           True
4          False
           ...  
2806516     True
2806517    False
2806518    False
2806519    False
2806520     True
Name: Stepinfo_endoftrackflag, Length: 2806521, dtype: bool
0          False
1          False
2          False
3           True
4          False
           ...  
2806516     True
2806517    False
2806518    False
2806519    False
2806520     True
Name: Stepinfo_endoftrackflag, Length: 2806521, dtype: bool


In [12]:
filt_x_min = x_pos_filtered.min()
filt_x_max = x_pos_filtered.max()
filt_y_min = y_pos_filtered.min()
filt_y_max = y_pos_filtered.max()
filt_z_min = z_pos_filtered.min()
filt_z_max = z_pos_filtered.max()

print(filt_x_min)
print(filt_x_max)
print(filt_y_min)
print(filt_y_max)
print(filt_z_min)
print(filt_z_max)

filt_positions_bins = (filt_z_max - filt_z_min)/200
print(filt_positions_bins)

-48000.0
48000.0
-48000.0
48000.0
-48000.0
48000.0
480.0


In [13]:
positions_mm = pony_data["distances"]
positions_min = positions_mm.min()
positions_max = positions_mm.max()
positions_bins = (positions_max - positions_min)/100

print(positions_bins)
print(positions_min)
print(positions_max)

827.3138705117536
160.49686239781204
82891.88391357317


### Polarization

In [14]:
polarization_x = pony_data["Stepinfo_polarization_X"]
polarization_y = pony_data["Stepinfo_polarization_Y"]
polarization_z = pony_data["Stepinfo_polarization_Z"]

polarization_x_min = polarization_x.min()
polarization_x_max = polarization_x.max()
polarization_y_min = polarization_y.min()
polarization_y_max = polarization_y.max()
polarization_z_min = polarization_z.min()
polarization_z_max = polarization_z.max()

polarization_bins = (polarization_x_max - polarization_y_min)/0.05
print(polarization_bins)

39.99997961980286


In [15]:
abs_polar_x = abs(polarization_x)
abs_polar_y = abs(polarization_y)
abs_polar_z = abs(polarization_z)

In [16]:
x_pol_filtered = abs_polar_x[mask]
y_pol_filtered = abs_polar_y[mask]
z_pol_filtered = abs_polar_z[mask]

In [17]:
pony_data["polarization"] = np.sqrt(x_pol_filtered**2 + y_pol_filtered**2 + z_pol_filtered**2)
pony_data = pony_data[pony_data["polarization"] > 0]

polarization = pony_data["polarization"]
polarization_min = polarization.min()
polarization_max = polarization.max()
print(polarization_min)
print(polarization_max)

0.9999999999999997
1.0000000000000002


# Plotting

## Parameters

In [18]:
TOOLTIPS = [
    ("Photon Energy", "$x"),
    ("Absorption Length", "$y"),
]

p = figure(width=1000, height=500, title="Absorption Length", x_axis_label='Photon Energy (eV)', y_axis_label='Absorption Length (m)', tooltips = TOOLTIPS)

p.scatter(energy_array, absorption_array, size=10, color="navy", alpha=0.5, legend_label = 'Pure Sea Water')
p.scatter(energy_array, glass_abs_array, size=10, color="red", alpha=0.5, legend_label = 'Borosilicate')

show(p)

In [19]:
TOOLTIPS = [
    ("Photon Energy", "$x"),
    ("Mie Scattering Length", "$y"),
]

p = figure(width=1000, height=500, title="Mie Scattering Length in Pure Sea Water", x_axis_label='Photon Energy (eV)', y_axis_label='Scattering Length (m)', tooltips = TOOLTIPS)

p.scatter(energy_array, mie_scat_array, size=10, color="navy", alpha=0.5)

show(p)

In [20]:
TOOLTIPS = [
    ("Photon Energy", "$x"),
    ("Rayleigh Scattering Length", "$y"),
]

p = figure(width=1000, height=500, title="Rayleigh Scattering Length in Pure Sea Water", x_axis_label='Photon Energy (eV)', y_axis_label='Scattering Length (m)', tooltips = TOOLTIPS)

p.scatter(energy_array, ray_interp, size=10, color="navy", alpha=0.5)
#p.scatter(ray_energy, ray_scattering, size=10, color="yellow", alpha=0.5)

show(p)

In [21]:
TOOLTIPS = [
    ("Photon Energy", "$x"),
    ("Refractive Index", "$y"),
]

p = figure(width=1000, height=500, title="Index of Refraction in Sea Water", x_axis_label='Photon Energy (eV)', y_axis_label='Refractive Index', tooltips = TOOLTIPS)

p.scatter(energy_array, index_interp, size=10, color="navy", alpha=0.5, legend_label = 'Pure Sea Water')
p.scatter(energy_array, glass_index_array, size=10, color="red", alpha=0.5, legend_label = 'Borosilicate')

show(p)

## Oscilloscope Data

In [67]:
TOOLTIPS = [
    ("Time", "$x"),
    ("Voltage", "$y"),
]

p = figure(width=1000, height=500, title="Oscilloscope", x_axis_label='Time (um)', y_axis_label='Voltage (V)', tooltips = TOOLTIPS)

p.line(time, smooth_voltage, color="navy", line_width = 1.5)

show(p)

## ROOT

### Positions

In [22]:
TOOLTIPS = [
    ("Position", "$x"),
    ("Events", "$y"),
]

p = figure(width=670, height=400, title="Distribution of Step Distances from Origin", y_axis_type="log", x_axis_label='Distance from Origin (mm)', y_axis_label='Number of Events / (100 mm)', tooltips = TOOLTIPS)

bins = np.linspace(positions_min, positions_max, 131)
hist, edges = np.histogram(positions_mm, density=False, bins=bins)
p.quad(top=hist, bottom=1, left=edges[:-1], right=edges[1:],
         fill_color="gray", line_color="gray", legend_label="Position",)
show(p)

In [28]:
TOOLTIPS = [
    ("Position", "$x"),
    ("Events", "$y"),
]

p = figure(width=670, height=400, title="Distribution of Step Distances from Origin", y_axis_type="log", x_axis_label='Distance from Origin (mm)', y_axis_label='Number of Events / (200 mm)', tooltips = TOOLTIPS)

bins = np.linspace(filt_z_min, filt_z_max, 300)
hist1, edges1 = np.histogram(x_pos_filtered, density=False, bins=bins)
#p.quad(top=hist1, bottom=1, left=edges1[:-1], right=edges1[1:],
         #fill_color="red", line_color="red", legend_label="X Positions", line_alpha = 0.20, fill_alpha = 0.15)
hist2, edges2 = np.histogram(y_pos_filtered, density=False, bins=bins)
p.quad(top=hist2, bottom=1, left=edges2[:-1], right=edges2[1:],
         fill_color="green", line_color="green", legend_label="Y Positions", line_alpha = 0.20, fill_alpha = 0.15)
hist3, edges3 = np.histogram(z_pos_filtered, density=False, bins=bins)
#p.quad(top=hist3, bottom=1, left=edges3[:-1], right=edges3[1:],
         #fill_color="blue", line_color="blue", legend_label="Z Positions", line_alpha = 0.20, fill_alpha = 0.15)

#p.output_backend = "svg"
#export_svgs(p, filename="flux_histogram.svg")
show(p)

#### Three Dimensions

In [24]:
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(z_pos_filtered, x_pos_filtered, y_pos_filtered, color = 'navy', s = 0.25)

ax.set_xlabel('Z Positions (m)')
ax.set_ylabel('X Positions (m)')
ax.set_zlabel('Y Positions (m)')
ax.set_title('3D Positions Along the Cylinder of Water')

ax.legend()
plt.show()

/var/folders/dy/04yh7cvx6rn6_64k0jg71wmc001d85/T/ipykernel_98259/2564459878.py:11: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend()


### Polarization

In [43]:
TOOLTIPS = [
    ("Polarization", "$x"),
    ("Number of Events", "$y"),
]

p = figure(width=670, height=400, title="Distribution of Polarization Values", x_axis_label='Polarization (C/m²)', y_axis_label='Number of Events / (0.05 C/m²)', tooltips = TOOLTIPS)

bins = np.linspace(0, polarization_x_max, 37)
hist1, edges1 = np.histogram(x_pol_filtered, density=False, bins=bins)
p.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
         fill_color="red", line_color="red", legend_label="Polarization in the X-Direction", line_alpha = 0.20, fill_alpha = 0.15)
hist2, edges2 = np.histogram(y_pol_filtered, density=False, bins=bins)
p.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:],
         fill_color="green", line_color="green", legend_label="Polarization in the Y-Direction", line_alpha = 0.20, fill_alpha = 0.15)
hist3, edges3 = np.histogram(z_pol_filtered, density=False, bins=bins)
p.quad(top=hist3, bottom=0, left=edges3[:-1], right=edges3[1:],
         fill_color="blue", line_color="blue", legend_label="Polarization in the Z-Direction", line_alpha = 0.20, fill_alpha = 0.15)

#p.output_backend = "svg"
#export_svgs(p, filename="flux_histogram.svg")
show(p)

In [30]:
TOOLTIPS = [
    ("Distance", "$x"),
    ("Polarization", "$y"),
]

p = figure(width=1000, height=500, x_axis_label='Distance from Origin on the Y-Axis (mm)', y_axis_label='Polarization (C/m²)', tooltips = TOOLTIPS)

#p.scatter(z_pos_filtered, x_pol_filtered, size=1, legend_label="Polarization in the X-Direction", color="red", alpha=0.5)
#p.scatter(z_pos_filtered, y_pol_filtered, size=1, legend_label="Polarization in the Y-Direction", color="green", alpha=0.5)
p.scatter(z_pos_filtered, z_pol_filtered, size=1, legend_label="Polarization in the Z-Direction", color="blue", alpha=0.5)
#p.scatter(z_pos_filtered, polarization, color="orange", legend_label="Total Polarization")

show(p)